In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import os
os.listdir('/kaggle/input/coleridgeinitiative-show-us-the-data')

Data provided to us in this competition comprises of following 4 items:

- train.csv - This file comprises of publication-id and the associated data title and labels along with the cleansed label.
- train folder - This folder provides json file for each of the id's present in the above csv file, with each of the json file describing section-details of the publication and the associated text with it.
- sample_submission.csv - This file comprises of id, for which we must predict the data-set title.
- test folder - This folder provides json file for each of the id's present in test.csv file and the json file describes the section details of the publication and the associated text.

In [ ]:
import pandas as pd
df_train=pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/train.csv')
df_train.head()

In [ ]:
df_train.shape

In [ ]:
print("Unique Values in each column of the data:\n")
for i in df_train.columns:
    print(str(i)+":" , df_train[i].nunique())
    

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train['dataset_title'].value_counts()

In [ ]:
df_train['dataset_label'].value_counts()

In [ ]:
df_test=pd.read_csv('/kaggle/input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
df_test

In [ ]:
df_input = pd.DataFrame(columns=['id','section_title','text','data_label'])
ids=df_train['Id'].values
import warnings
warnings.filterwarnings("ignore")
for id in ids:
    df=pd.read_json ('../input/coleridgeinitiative-show-us-the-data/train/{}.json'.format(id))
    for data_label in df_train[df_train['Id']==id]['dataset_label'].values:
        new_df=df[df['text'].str.contains(data_label)].copy(deep=True)
        new_df.loc[:,['data_label']] = data_label
        new_df.loc[:,['id']] = id
        new_df.reset_index(inplace=True,drop=True)
        df_input=pd.concat([df_input, new_df], ignore_index=True)
        df_input.reset_index(inplace=True,drop=True)



In [ ]:
df_input

In [ ]:
# df_input["section_title"][53043]

In [ ]:
print("Unique Values in each column of the data:\n")
for i in df_input.columns:
    print(str(i)+":" , df_input[i].nunique())
    

In [ ]:
df_input.shape

In [ ]:
df_input.to_csv("df_input.csv")

In [ ]:
df=pd.read_csv("./df_input.csv",index_col="Unnamed: 0")
df.head()

### EDA on the given data

In [ ]:
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import time
import pickle
import pyLDAvis.sklearn
from collections import Counter
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.autonotebook import tqdm
import string
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
#Tensorflow Libraries


STOPWORDS = set(stopwords.words('english'))

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Input

import re

In [ ]:
words =list( df_input['data_label'].values)
words=[word.split() for word in words]

In [ ]:
allwords = []
for wordlist in words:
    allwords += wordlist
# print(allwords)

In [ ]:
mostcommon = FreqDist(allwords).most_common(100)
mostcommon

In [ ]:
wordcloud = WordCloud(width=1600, height=800, background_color='white').generate(str(mostcommon))
fig = plt.figure(figsize=(30,10), facecolor='white')
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('Top 100 Most Common Words in Data Label\n', fontsize=50)
plt.tight_layout(pad=0)
plt.show()

In [ ]:
mostcommon_small = FreqDist(allwords).most_common(25)
x, y = zip(*mostcommon_small)
plt.figure(figsize=(50,30))
plt.margins(0.02)
plt.bar(x, y)
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Freq of 25 Most Common Words in Data-Label', fontsize=60)
plt.show()

In [ ]:
words =list( df_input['section_title'].values)
stopwords=['ourselves', 'hers','the', 'between', 'yourself', 'but', 'again','of', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']
split_words=[]
for word in words:
    lo_w=[]
    list_of_words=str(word).split()
    for w in list_of_words:
        if w not in stopwords:
            lo_w.append(w)
    split_words.append(lo_w)
allwords = []
for wordlist in split_words:
    allwords += wordlist
mostcommon = FreqDist(allwords).most_common(100)
wordcloud = WordCloud(width=1600, height=800, background_color='white').generate(str(mostcommon))
fig = plt.figure(figsize=(30,10), facecolor='white')
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('Top 100 Most Common Words in Section Title', fontsize=50)
plt.tight_layout(pad=0)
plt.show()
#print(allwords)
mostcommon_small = FreqDist(allwords).most_common(25)
x, y = zip(*mostcommon_small)
plt.figure(figsize=(50,30))
plt.margins(0.02)
plt.bar(x, y)
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Freq of 25 Most Common Words in Section-Title', fontsize=60)
plt.show()

### Model Building

In [ ]:
#At first we need to clean the data
import warnings
warnings.filterwarnings("ignore")

df_input.head()

In [ ]:
#Cleaning the text


def cleantext(data,column):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    tweets_clean = []
    t=-1
    for i in data[column]:
        print()
        i = str(i).lower()
        i = re.sub('\[.*?\]', '', i)
#         i = re.sub('https?://\S+|www\.\S+', '', i)
#         i = re.sub('<.*?>+', '', i)
#         i = re.sub('[%s]' % re.escape(string.punctuation), '', i)
        i = re.sub('\n', '', i)
#         i = re.sub('\w*\d\w*', '', i)
#         i = re.sub(r'\$\w*', '', i)
        i= re.sub('[^a-zA-Z]',' ',i)
            # remove old style retweet text "RT"
#         i = re.sub(r'^RT[\s]+', '', i)
            # remove hyperlinks
#         i = re.sub(r'https?:\/\/.*[\r\n]*', '', i)
            # remove hashtags
            # only removing the hash # sign from the word
        i = re.sub(r'[#@]+', '', i)
    
    
        tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                                       reduce_len=True)
        tweet_tokens = tokenizer.tokenize(i)
        
            
            
        text = [word for word in tweet_tokens if word not in STOPWORDS]
        text = ' '.join(text)
        tweets_clean.append(text)
        t+=1
        print(f"{t}th sentence cleaning completed / {len(data[column])} ")
           
    

    
    return tweets_clean

In [ ]:
tqdm.pandas()
clean_text= cleantext(df_input,"text")
cleaned_text= pd.DataFrame(clean_text, columns=["cleaned_text"])
cleaned_text.head()
%time

In [ ]:
df_input.insert(3,"cleaned_text",cleaned_text["cleaned_text"])

In [ ]:
df_input.head()

In [ ]:
## Splitting the data into dependent and independent variables to apply model.

X= df_input["cleaned_text"]
y= df_input.data_label.astype(str)

In [ ]:
y.nunique()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_label=le.fit_transform(y)
df_input["y_label"]= y_label
df_input.head()

In [ ]:
#Arranging data_Label and y_label in the form of dictionary
lis={}
xip=zip(df_input["data_label"], df_input["y_label"])
for a,b in xip:
    if a not in lis.keys():
        lis[a] = b
    else:
        pass

lis

In [ ]:
#Re-Arranging the dictionary in the ascending order
labels={}
for k,v in lis.items():
    labels[v]= k
    
y_dummy_labels={}
for i in sorted(labels):
    y_dummy_labels[i] = labels[i]
    
y_dummy_labels
    

In [ ]:
from tensorflow.keras.utils import to_categorical

Y=to_categorical(y_label,num_classes=108)
Y


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional,Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
# Initializing Hyper params

embedding_dim = 300
vocab_size=10000
padding_type='post'
trunc_type='post'
oov_tok = "<OOV>"
max_length=200

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok) 
tokenizer.fit_on_texts(X)  
word_index = tokenizer.word_index
# print(word_index)


In [ ]:
x = tokenizer.texts_to_sequences(X) 
x = pad_sequences(x, padding = padding_type, maxlen=max_length) 
# print(len(train_sequences[0]))
# print(len(train_padded[0]))
print(x)

In [ ]:
for i in range(0,4):
    print("sentence: {0}\nsequences: {1} ".format(X.iloc[i],x[i]))


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, Y, test_size=0.30, random_state=1)
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_test: {y_test.shape}")

In [ ]:
## Creating model
# Initializing Hyper params

embedding_dim = 300
vocab_size=10000

model1=Sequential()
model1.add(Embedding(vocab_size,embedding_dim,input_length=max_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(256,activation = 'relu'))
model1.add(Dense(108,activation='softmax')) # We have chose 108 nodes because oof the number of unique values in the y_label
model1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
tf.keras.utils.plot_model(model1)


In [ ]:
tf.keras.utils.plot_model(model1, show_shapes=True, dpi=48)

In [ ]:
# To avoid overfitting the data we use EarlyStopping

## To avoid overfitting the data we use EarlyStopping

learning_rate_reduction=ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.5,min_lr=0.00001)
early_stopping= EarlyStopping(monitor='val_loss',min_delta=0, patience=2)
num_epochs = 30
history = model1.fit(X_train,y_train,batch_size=128,epochs=num_epochs,
          validation_split=0.2,callbacks=[learning_rate_reduction,early_stopping])

In [ ]:
fig=plt.figure(figsize=(10,5))
plt.title("MODEL PERFORMANCE DURING TRAINING AND EVALUATION")
fig1=fig.add_subplot(121)
plt.plot(history.history['accuracy'],color='Green')
plt.plot(history.history["val_accuracy"],color='red')
# plt.ylim(0.90, 1.05)
plt.xlabel("epoch")
plt.ylabel("accuracies")
plt.legend(['Train accuracy','Validation Accuracy'])

fig2=fig.add_subplot(122)
plt.plot(history.history["loss"],color='g')
plt.plot(history.history["val_loss"],color='r')
# plt.ylim(0.0, 1)
plt.xlabel("epoch")
plt.ylabel("Losses")
plt.legend(['Train_Loss','Val_Loss'])

plt.savefig("Model Performance.jpg")
plt.show()

In [ ]:
%%time
score = model1.evaluate(X_test, y_test, batch_size=128)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

In [ ]:
predict=np.argmax(model1.predict(X_test), axis=-1)
predict

In [ ]:
l=[]
for i in predict:
    l.append(y_dummy_labels[i])
    
pred_labels= np.array(l)
# pred_label= y_dummy_labels[predict]

In [ ]:
## Evaluating with the performance metrics (JACCARD RULE)



# Evaluate it using the metric that they use in this dataset
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
scores = []
for i in range(pred_labels.shape[0]):
    pred =str(pred_labels[i])
    true = str(y_dummy_labels[np.argmax(y_test[i], axis=-1)])
    scores.append(jaccard(pred, true))

print(f'Score: {np.mean(scores)}')
# print(scores)

In [ ]:
# Saving the model
# model1.save("../input/BI-LSTM.h5")
model1.save("./BI-LSTM.h5")

In [ ]:
#For better score, need to fine tune the model layers & its hyper parameters.